# ISYE/Math/CS 425 - Assignment 4
### Bryan Luu

All code in Julia version 1.5.2

## Helper functions

In [1]:
to_sym(v) = map(x->Symbol(x), v); # function to convert string into array of symbols

In [2]:
# calculate the flow into v given flows x
function flow(x, v)
    E = keys(x);
    inflow = sum(Float64[x[(i,j)] for (i,j) in E if j==v]);
    outflow = sum(Float64[x[(i,j)] for (i,j) in E if i==v]);
    inflow - outflow
end

flow (generic function with 1 method)

In [7]:
# Bread-first-search for (u,v)-path in the graph defined by adjacency list L
function BFS(L, u, v)
    V = keys(L);
    p = Dict{Any, Any}(zip(V, -1*ones(length(V))));
    p[u] = 0;
    q = [u]; # queue for BFS
    while !isempty(q)
        n = popfirst!(q);
        if n == v
            break
        end
        for (m, rev) in L[n]
            if p[m] == -1
                push!(q, m);
                p[m] = n;
            end
        end
    end
    i = v;
    path = [];
    while(i != u)
        if p[i] == -1
            return []
        end
        pushfirst!(path, i);
        i = p[i];
    end
    pushfirst!(path, u);
    path
end

BFS (generic function with 1 method)

In [9]:
# Augmenting path algorithm for graph G=(V,E), capacities u, and start r and end s.
function APA(G, u, r=:r, s=:s; verbose=false)
    V, E = G;
    
    # Initialize adjacency list L of augmented graph G' = G
    L = Dict{Any, Any}(zip(V, [[] for v in V]));
    for (u, v) in E
        push!(L[u], (v, false)); # the boolean represents whether the edge was reversed
    end
    
    # Construct flow vector
    x = Dict(zip(E, zeros(length(E))));
    
    path = BFS(L, r, s);
    i = 1;
    while(!isempty(path))
        if verbose
            println("---Iteration $(i)---")
        end
        # find x-width
        arcs = [p for p in zip(path[1:end-1], path[2:end])];
        eps1 = min(Inf, [u[(i,j)] - x[(i,j)] for (i,j) in arcs if (j, false) in L[i]]...);
        eps2 = min(Inf, [x[(j,i)] for (i,j) in arcs if (j, true) in L[i]]...);
        x_width = min(eps1, eps2);
        
        if verbose
            println(path)
            println("x-width: $(x_width)")
            println("e\tx[e]")
        end

        if x_width == Inf
            break
        end

        for e in arcs
            if e in E
                x[e] += x_width; # increase flow on forward arcs
            else
                x[reverse(e)] -= x_width; # decrease flow on reverse arcs
            end
        end

        if verbose
            for e in E
                println("$(e)\t$(x[e])")
            end
        end

        # construct augmented graph as adjacency list
        L = Dict{Symbol, Any}(zip(V, [[] for v in V])); # Adjacency list
        for (v, w) in E
            if x[(v, w)] < u[(v, w)]
                push!(L[v], (w, false)); # add a forward arc
            end
            if x[(v, w)] > 0
                push!(L[w], (v, true)); # add a reverse arc
            end
        end

        if verbose
            println("G':\n$(join(["$v\t$(L[v])" for v in V], "\n"))")
        end

        path = BFS(L, r, s);
        i += 1;
    end
    
    max_flow = flow(x, s);
                                    
    if verbose
        println("===== OPTIMUM FOUND =====")
        println("Max-flow: $(max_flow)")
    end
                                    
    return (max_flow, x);
end

APA (generic function with 3 methods)

## Exercise 2

Max-flow instance set-up for Exercise 2

In [5]:
V_str = "rpbacsqd"; # String representing all nodes
V = Set(to_sym(collect(V_str)));

L = Dict{Symbol, Any}(zip(V, [[] for v in V])); # Adjacency list
L[:r] = [:p, :a, :q];
L[:p] = [:q, :b];
L[:b] = [:a, :s];
L[:a] = [:c, :d];
L[:c] = [:q, :s, :b];
L[:q] = [:p, :b, :d];
L[:d] = [:c, :s];

E = Set(); # Edge Set
for i in V
    for j in L[i]
        push!(E, (i, j));
    end
end

G = (V, E); # graph

Lu = Dict{Symbol, Any}(zip(V, [[] for v in V])); # Capacities in Adjacency List form
Lu[:r] = [6, 9, 4];
Lu[:p] = [2, 3];
Lu[:b] = [1, 8];
Lu[:a] = [8, 1];
Lu[:c] = [1, 4, 2];
Lu[:q] = [1, 2, 6];
Lu[:d] = [1, 6];

u = Dict{Any, Float64}(); # Capacities
for v in V
    for (i, w) in enumerate(L[v])
        u[(v, w)] = Lu[v][i]
    end
end

Call the `APA` function to solve for max-flow on G and u

In [10]:
max_flow, x = APA(G, u, verbose=true);

---Iteration 1---
Any[:r, :p, :b, :s]
x-width: 3.0
e	x[e]
(:q, :p)	0.0
(:p, :b)	3.0
(:r, :p)	3.0
(:a, :c)	0.0
(:b, :a)	0.0
(:a, :d)	0.0
(:c, :b)	0.0
(:b, :s)	3.0
(:p, :q)	0.0
(:d, :c)	0.0
(:c, :q)	0.0
(:c, :s)	0.0
(:r, :a)	0.0
(:r, :q)	0.0
(:d, :s)	0.0
(:q, :b)	0.0
(:q, :d)	0.0
G':
a	Any[(:c, false), (:d, false)]
b	Any[(:p, true), (:a, false), (:s, false)]
p	Any[(:r, true), (:q, false)]
d	Any[(:c, false), (:s, false)]
s	Any[(:b, true)]
c	Any[(:b, false), (:q, false), (:s, false)]
r	Any[(:p, false), (:a, false), (:q, false)]
q	Any[(:p, false), (:b, false), (:d, false)]
---Iteration 2---
Any[:r, :a, :c, :s]
x-width: 4.0
e	x[e]
(:q, :p)	0.0
(:p, :b)	3.0
(:r, :p)	3.0
(:a, :c)	4.0
(:b, :a)	0.0
(:a, :d)	0.0
(:c, :b)	0.0
(:b, :s)	3.0
(:p, :q)	0.0
(:d, :c)	0.0
(:c, :q)	0.0
(:c, :s)	4.0
(:r, :a)	4.0
(:r, :q)	0.0
(:d, :s)	0.0
(:q, :b)	0.0
(:q, :d)	0.0
G':
a	Any[(:c, false), (:d, false), (:r, true)]
b	Any[(:p, true), (:a, false), (:s, false)]
p	Any[(:r, true), (:q, false)]
d	Any[(:c, false), (:s,

Therefore the maximum-flow is 17, which is also the minimum-cut.

## Exercise 5

Max-flow instance set-up for Exercise 5, letting _P = {a,b,c,d,e,f,g,h,i,j}_ and _Q = {A,B,C,D,E,F,G,H,I,J}_.

In [11]:
P_str = "abcdefghij"; # Specify P here as a string
Q_str = uppercase(P_str); # Specify Q here as a string

P, Q = to_sym(collect(P_str)), to_sym(collect(Q_str)); # creates arrays for P and Q

V = Set([P..., Q..., :r, :s]); # Node Set

L = Dict{Symbol, Any}(zip(V, [[] for v in V])); # Adjacency list
L[:a] = [:A, :C, :D];
L[:b] = [:A, :B, :H];
L[:c] = [:A, :C];
L[:d] = [:C, :D];
L[:e] = [:C, :D];
L[:f] = [:B, :E, :G];
L[:g] = [:E, :F, :J];
L[:h] = [:E, :F, :G, :H, :J];
L[:i] = [:I];
L[:j] = [:D, :I];

L[:r] = P;
for v in Q
    L[v] = [:s];
end

E = Set(); # Edge Set
for i in keys(L)
    for j in L[i]
        push!(E, (i, j));
    end
end

G = (V, E); # graph

u = Dict(zip(E, Inf*ones(length(E)))); # Capacities
for p in P
    u[(:r, p)] = 1;
end
for q in Q
    u[(q, :s)] = 1;
end

Run APA (augmenting path algorithm) to find max matching

In [13]:
# x is now the max-flow
max_flow, x = APA(G, u, verbose=true);
# get matchings from flows
M = [(p, q) for (p, q) in keys(x) if x[(p, q)] == 1 && (in(p, P) || in(q, Q))];
println("Matchings:")
println(join(sort(M), "\n"))

---Iteration 1---
Any[:r, :b, :B, :s]
x-width: 1.0
e	x[e]
(:r, :b)	1.0
(:r, :d)	0.0
(:b, :B)	1.0
(:e, :C)	0.0
(:c, :C)	0.0
(:h, :H)	0.0
(:r, :i)	0.0
(:g, :F)	0.0
(:h, :F)	0.0
(:a, :D)	0.0
(:g, :J)	0.0
(:h, :E)	0.0
(:h, :J)	0.0
(:g, :E)	0.0
(:h, :G)	0.0
(:b, :A)	0.0
(:r, :j)	0.0
(:d, :C)	0.0
(:r, :a)	0.0
(:G, :s)	0.0
(:A, :s)	0.0
(:B, :s)	1.0
(:C, :s)	0.0
(:e, :D)	0.0
(:H, :s)	0.0
(:f, :E)	0.0
(:f, :G)	0.0
(:F, :s)	0.0
(:D, :s)	0.0
(:E, :s)	0.0
(:c, :A)	0.0
(:f, :B)	0.0
(:J, :s)	0.0
(:a, :C)	0.0
(:r, :h)	0.0
(:i, :I)	0.0
(:b, :H)	0.0
(:r, :g)	0.0
(:j, :I)	0.0
(:a, :A)	0.0
(:r, :c)	0.0
(:r, :e)	0.0
(:j, :D)	0.0
(:r, :f)	0.0
(:d, :D)	0.0
(:I, :s)	0.0
G':
j	Any[(:I, false), (:D, false)]
D	Any[(:s, false)]
d	Any[(:C, false), (:D, false)]
A	Any[(:s, false)]
g	Any[(:F, false), (:J, false), (:E, false)]
r	Any[(:d, false), (:i, false), (:j, false), (:a, false), (:h, false), (:g, false), (:c, false), (:e, false), (:f, false)]
a	Any[(:D, false), (:C, false), (:A, false)]
J	Any[(:s, false)]
i	Any[

(:h, :G)	0.0
(:b, :A)	0.0
(:r, :j)	1.0
(:d, :C)	1.0
(:r, :a)	1.0
(:G, :s)	0.0
(:A, :s)	1.0
(:B, :s)	1.0
(:C, :s)	1.0
(:e, :D)	0.0
(:H, :s)	1.0
(:f, :E)	1.0
(:f, :G)	0.0
(:F, :s)	1.0
(:D, :s)	1.0
(:E, :s)	1.0
(:c, :A)	0.0
(:f, :B)	0.0
(:J, :s)	0.0
(:a, :C)	0.0
(:r, :h)	1.0
(:i, :I)	1.0
(:b, :H)	0.0
(:r, :g)	1.0
(:j, :I)	0.0
(:a, :A)	1.0
(:r, :c)	0.0
(:r, :e)	0.0
(:j, :D)	1.0
(:r, :f)	1.0
(:d, :D)	0.0
(:I, :s)	1.0
G':
j	Any[(:r, true), (:I, false), (:D, false)]
D	Any[(:j, true)]
d	Any[(:r, true), (:C, false), (:D, false)]
A	Any[(:a, true)]
g	Any[(:F, false), (:J, false), (:E, false), (:r, true)]
r	Any[(:c, false), (:e, false)]
a	Any[(:D, false), (:r, true), (:C, false), (:A, false)]
J	Any[(:s, false)]
i	Any[(:r, true), (:I, false)]
b	Any[(:r, true), (:B, false), (:A, false), (:H, false)]
F	Any[(:g, true)]
e	Any[(:C, false), (:D, false)]
B	Any[(:b, true)]
c	Any[(:C, false), (:A, false)]
h	Any[(:H, false), (:F, false), (:E, false), (:J, false), (:G, false), (:r, true)]
E	Any[(:f, true)]
s	

So the maximum-matching is above, with a cardinality of $|M| = 8$, which is also the minimum-cover.